In [1]:
from catboost import CatBoost, CatBoostClassifier
import lightgbm as lgb

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn import ensemble,metrics,cross_validation,preprocessing,grid_search
path='C:/Projects/Hackerearth_2/'

C:\Users\sharm_000\Anaconda2\envs\py36\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
data=pd.read_csv(path+'train.csv',dtype={'roadId':object})
labels=pd.read_csv(path+'labels.csv',dtype={'roadId':object})
test_data=pd.read_csv(path+'test.csv',dtype={'roadId':object})
print (data.shape,labels.shape)
len(np.unique(data['roadId'])),len(np.unique(test_data['roadId']))

(28914, 14) (4825, 3)


(4825, 400)

In [4]:
new_data=pd.merge(data,labels,on='roadId',how='left')
new_data.columns=[i.strip(' ')for i in new_data.columns]
# new_data['roadCoordinates_count']=new_data['roadCoordinates'].apply(lambda x: len(x.split()))
new_data['laneLineCoordinates_count']=new_data['laneLineCoordinates'].apply(lambda x: len(x.split()))
new_data=new_data.fillna(0)
new_data.head(1)

,roadId,totalLaneLines,laneLineId,laneLineCoordinates,totalLinesOnLeft,totalLaneLinesOnRight,distFromLaneLineOnLeft,distFromLaneLineOnRight,laneLineLength,roadLength,noOfIntersectingLaneLinesLeft,noOfIntersectingLaneLinesRight,isIntersectingWithRoadGeometry,roadCategory,roadCoordinates,noOfLanes,laneLineCoordinates_count
0,3177248677,3,X3177248677,11.117636626741222 49.43799310961545|11.11761...,2,0,3.863433276,,4.396337,40.57071,0,0,false,2,11.11747 49.43763|11.11753 49.43776|11.11761 ...,2,3


In [5]:
def num_con(x):
    try:
        return float(x)
    except:
        return 0
    
def geom(x):
    if x=='false':
        return 0
    else:
        return 1

In [6]:
new_data['distFromLaneLineOnLeft']=new_data['distFromLaneLineOnLeft'].apply(lambda x: num_con(x))
new_data['distFromLaneLineOnRight']=new_data['distFromLaneLineOnRight'].apply(lambda x: num_con(x))
new_data['isIntersectingWithRoadGeometry']=new_data['isIntersectingWithRoadGeometry'].apply(lambda x: num_con(x))

In [7]:
new_data.columns

Index(['roadId', 'totalLaneLines', 'laneLineId', 'laneLineCoordinates',
       'totalLinesOnLeft', 'totalLaneLinesOnRight', 'distFromLaneLineOnLeft',
       'distFromLaneLineOnRight', 'laneLineLength', 'roadLength',
       'noOfIntersectingLaneLinesLeft', 'noOfIntersectingLaneLinesRight',
       'isIntersectingWithRoadGeometry', 'roadCategory', 'roadCoordinates',
       'noOfLanes', 'laneLineCoordinates_count'],
      dtype='object')

In [8]:
train_pro_data=new_data.groupby(by='roadId').agg({'laneLineId':{'laneLineId':'nunique'},
        'totalLaneLines':{'totalLaneLines_sum':'sum','totalLaneLines_mean':'mean','totalLaneLines_min':'min','totalLaneLines_max':'max'},\
        'distFromLaneLineOnRight':{'distFromLaneLineOnRight_mean':'mean','distFromLaneLineOnRight_min':'min',\
        'distFromLaneLineOnRight_sum':'sum',  'distFromLaneLineOnRight_max':'max'},\
        'distFromLaneLineOnLeft':{'distFromLaneLineOnLeft_mean':'mean','distFromLaneLineOnLeft_min':'min',\
        'distFromLaneLineOnLeft_sum':'sum',                   'distFromLaneLineOnLeft_max':'max'},\
        'laneLineLength':{'laneLineLength_mean':'mean','laneLineLength_min':'min','laneLineLength_max':'max','laneLineLength_sum':'sum',},\
        'roadLength':{'roadLength_mean':'mean','roadLength_min':'min','roadLength_max':'max'},\
        'noOfIntersectingLaneLinesLeft':{'noOfIntersectingLaneLinesLeft_sum':'sum',\
        'noOfIntersectingLaneLinesLeft_min':'min','noOfIntersectingLaneLinesLeft_max':'max',\
        'noOfIntersectingLaneLinesLeft_mean':'mean'},\
        'noOfIntersectingLaneLinesRight':{'noOfIntersectingLaneLinesRight_sum':'sum',\
                                          'noOfIntersectingLaneLinesRight_min':'min',\
                                          'noOfIntersectingLaneLinesRight_max':'max'},\
        'roadCategory':{'roadCategory':'mean'},'isIntersectingWithRoadGeometry':{'isIntersectingWithRoadGeometry':'sum'},\
        'laneLineCoordinates_count':{'laneLineCoordinates_count':'mean','laneLineCoordinates_min':'min','laneLineCoordinates_max':'max',\
         'laneLineCoordinates_sum':'sum'},'noOfLanes':{'noOfLanes':'mean'}}).reset_index()

train_pro_data.columns=train_pro_data.columns.droplevel(0)
train_pro_data.rename(columns={'':'roadId'},inplace=True)

In [9]:
train_pro_data.head()

,roadId,nunique,totalLaneLines_sum,totalLaneLines_mean,totalLaneLines_min,totalLaneLines_max,distFromLaneLineOnRight_mean,distFromLaneLineOnRight_min,distFromLaneLineOnRight_sum,distFromLaneLineOnRight_max,...,noOfIntersectingLaneLinesRight_sum,noOfIntersectingLaneLinesRight_min,noOfIntersectingLaneLinesRight_max,roadCategory,isIntersectingWithRoadGeometry,laneLineCoordinates_count,laneLineCoordinates_min,laneLineCoordinates_max,laneLineCoordinates_sum,noOfLanes
0,1003085808,5,25,5.0,5,5,2.118718,0.0,10.593588,3.653125,...,0,0,0,2,0,8.600000,4,14,43,2
1,1003388168,3,9,3.0,3,3,2.913640,0.0,8.740921,4.690038,...,0,0,0,2,0,6.333333,5,9,19,2
2,1003388169,4,16,4.0,4,4,3.167543,0.0,12.670172,4.901373,...,0,0,0,2,0,4.750000,3,7,19,2
3,1009949115,4,16,4.0,4,4,2.858797,0.0,11.435189,3.888012,...,0,0,0,1,0,13.500000,10,16,54,3
4,1009975629,4,16,4.0,4,4,2.939509,0.0,11.758036,3.952289,...,0,0,0,1,0,5.250000,4,6,21,3


In [10]:
train_pro_data.columns

Index(['roadId', 'nunique', 'totalLaneLines_sum', 'totalLaneLines_mean',
       'totalLaneLines_min', 'totalLaneLines_max',
       'distFromLaneLineOnRight_mean', 'distFromLaneLineOnRight_min',
       'distFromLaneLineOnRight_sum', 'distFromLaneLineOnRight_max',
       'distFromLaneLineOnLeft_mean', 'distFromLaneLineOnLeft_min',
       'distFromLaneLineOnLeft_sum', 'distFromLaneLineOnLeft_max',
       'laneLineLength_mean', 'laneLineLength_min', 'laneLineLength_max',
       'laneLineLength_sum', 'roadLength_mean', 'roadLength_min',
       'roadLength_max', 'noOfIntersectingLaneLinesLeft_sum',
       'noOfIntersectingLaneLinesLeft_min',
       'noOfIntersectingLaneLinesLeft_max',
       'noOfIntersectingLaneLinesLeft_mean',
       'noOfIntersectingLaneLinesRight_sum',
       'noOfIntersectingLaneLinesRight_min',
       'noOfIntersectingLaneLinesRight_max', 'roadCategory',
       'isIntersectingWithRoadGeometry', 'laneLineCoordinates_count',
       'laneLineCoordinates_min', 'laneLineCoo

In [11]:
train_pro_data['areaFromLaneLine_mean']=train_pro_data['distFromLaneLineOnRight_mean']*train_pro_data['distFromLaneLineOnLeft_mean']
train_pro_data['areaFromLaneLine_min']=train_pro_data['distFromLaneLineOnRight_min']*train_pro_data['distFromLaneLineOnLeft_min']
train_pro_data['areaFromLaneLine_sum']=train_pro_data['distFromLaneLineOnRight_sum']*train_pro_data['distFromLaneLineOnLeft_sum']
train_pro_data['areaFromLaneLine_max']=train_pro_data['distFromLaneLineOnRight_max']*train_pro_data['distFromLaneLineOnLeft_max']

In [12]:
train_pro_data['ratio_Intersect_sum']=train_pro_data['noOfIntersectingLaneLinesLeft_sum']/train_pro_data['noOfIntersectingLaneLinesRight_sum']
train_pro_data['ratio_Intersect_min']=train_pro_data['noOfIntersectingLaneLinesLeft_min']/train_pro_data['noOfIntersectingLaneLinesRight_min']
train_pro_data['ratio_Intersect_max']=train_pro_data['noOfIntersectingLaneLinesLeft_max']/train_pro_data['noOfIntersectingLaneLinesRight_max']

In [13]:
train_pro_data['noOfLanes']=train_pro_data['noOfLanes'].apply(lambda x:str(x))

In [14]:
lbl=preprocessing.LabelEncoder()
train_pro_data['noOfLanes']=lbl.fit_transform(train_pro_data['noOfLanes'])

In [15]:
# train_pro_data=pd.concat([train_pro_data,pd.get_dummies(train_pro_data['roadCategory'],prefix='rC',prefix_sep='_')[['rC_1','rC_2']]],axis=1)

In [16]:
# del train_pro_data['roadCategory']
# train_pro_data.shape

In [17]:
# new_test_data=pd.merge(test_data,labels,on='roadId',how='left')
test_data.columns=[i.strip(' ')for i in test_data.columns]
test_data.shape,len(np.unique(test_data['roadId']))

((2493, 14), 400)

In [18]:
test_data.head()

,roadId,totalLaneLines,laneLineId,laneLineCoordinates,totalLinesOnLeft,totalLaneLinesOnRight,distFromLaneLineOnLeft,distFromLaneLineOnRight,laneLineLength,roadLength,noOfIntersectingLaneLinesLeft,noOfIntersectingLaneLinesRight,isIntersectingWithRoadGeometry,roadCategory
0,291531296,5,X291531296,11.329899604343693 43.72956824243773|11.33004...,2,2,5.655398294,3.49899823,12.054597,12.056004,0,0,false,1
1,291531296,5,1291531296,11.329900151795446 43.729503478051974|11.3300...,4,0,3.685959755,,12.060141,12.056004,0,0,false,1
2,291531296,5,2291531296,11.329899913660833 43.72953696742431|11.33004...,3,1,3.49899823,3.685959755,12.052468,12.056004,0,0,false,1
3,291531296,5,3291531296,11.329899175813209 43.72961906004278|11.33004...,1,3,3.481580911,5.655398294,12.046642,12.056004,0,0,false,1
4,291531296,5,4291531296,11.329899073264464 43.729650480155186|11.3300...,0,4,,3.481580911,12.054345,12.056004,0,0,false,1


In [19]:
# new_test_data['roadCoordinates_count']=new_test_data['roadCoordinates'].apply(lambda x: len(x.split()))
test_data['laneLineCoordinates_count']=test_data['laneLineCoordinates'].apply(lambda x: len(x.split()))
test_data=test_data.fillna(0)
print (test_data.shape)
test_data.head(2)

(2493, 15)


,roadId,totalLaneLines,laneLineId,laneLineCoordinates,totalLinesOnLeft,totalLaneLinesOnRight,distFromLaneLineOnLeft,distFromLaneLineOnRight,laneLineLength,roadLength,noOfIntersectingLaneLinesLeft,noOfIntersectingLaneLinesRight,isIntersectingWithRoadGeometry,roadCategory,laneLineCoordinates_count
0,291531296,5,X291531296,11.329899604343693 43.72956824243773|11.33004...,2,2,5.655398294,3.49899823,12.054597,12.056004,0,0,false,1,3
1,291531296,5,1291531296,11.329900151795446 43.729503478051974|11.3300...,4,0,3.685959755,,12.060141,12.056004,0,0,false,1,3


In [20]:
test_data['distFromLaneLineOnLeft']=test_data['distFromLaneLineOnLeft'].apply(lambda x: num_con(x))
test_data['distFromLaneLineOnRight']=test_data['distFromLaneLineOnRight'].apply(lambda x: num_con(x))
test_data['isIntersectingWithRoadGeometry']=test_data['isIntersectingWithRoadGeometry'].apply(lambda x: num_con(x))

In [21]:
test_pro_data=test_data.groupby(by='roadId').agg({'laneLineId':{'laneLineId':'nunique'},
        'totalLaneLines':{'totalLaneLines_sum':'sum','totalLaneLines_mean':'mean','totalLaneLines_min':'min','totalLaneLines_max':'max'},\
        'distFromLaneLineOnRight':{'distFromLaneLineOnRight_mean':'mean','distFromLaneLineOnRight_min':'min',\
        'distFromLaneLineOnRight_sum':'sum',  'distFromLaneLineOnRight_max':'max'},\
        'distFromLaneLineOnLeft':{'distFromLaneLineOnLeft_mean':'mean','distFromLaneLineOnLeft_min':'min',\
        'distFromLaneLineOnLeft_sum':'sum',                   'distFromLaneLineOnLeft_max':'max'},\
        'laneLineLength':{'laneLineLength_mean':'mean','laneLineLength_min':'min','laneLineLength_max':'max','laneLineLength_sum':'sum',},\
        'roadLength':{'roadLength_mean':'mean','roadLength_min':'min','roadLength_max':'max'},\
        'noOfIntersectingLaneLinesLeft':{'noOfIntersectingLaneLinesLeft_sum':'sum',\
        'noOfIntersectingLaneLinesLeft_min':'min','noOfIntersectingLaneLinesLeft_max':'max',\
        'noOfIntersectingLaneLinesLeft_mean':'mean'},\
        'noOfIntersectingLaneLinesRight':{'noOfIntersectingLaneLinesRight_sum':'sum',\
                                          'noOfIntersectingLaneLinesRight_min':'min',\
                                          'noOfIntersectingLaneLinesRight_max':'max'},\
        'roadCategory':{'roadCategory':'mean'},'isIntersectingWithRoadGeometry':{'isIntersectingWithRoadGeometry':'sum'},\
        'laneLineCoordinates_count':{'laneLineCoordinates_count':'mean','laneLineCoordinates_min':'min','laneLineCoordinates_max':'max',\
         'laneLineCoordinates_sum':'sum'}}).reset_index()

test_pro_data.columns=test_pro_data.columns.droplevel(0)
test_pro_data.rename(columns={'':'roadId'},inplace=True)

In [22]:
test_pro_data['areaFromLaneLine_mean']=test_pro_data['distFromLaneLineOnRight_mean']*test_pro_data['distFromLaneLineOnLeft_mean']
test_pro_data['areaFromLaneLine_min']=test_pro_data['distFromLaneLineOnRight_min']*test_pro_data['distFromLaneLineOnLeft_min']
test_pro_data['areaFromLaneLine_sum']=test_pro_data['distFromLaneLineOnRight_sum']*test_pro_data['distFromLaneLineOnLeft_sum']
test_pro_data['areaFromLaneLine_max']=test_pro_data['distFromLaneLineOnRight_max']*test_pro_data['distFromLaneLineOnLeft_max']

In [23]:
test_pro_data['ratio_Intersect_sum']=test_pro_data['noOfIntersectingLaneLinesLeft_sum']/test_pro_data['noOfIntersectingLaneLinesRight_sum']
test_pro_data['ratio_Intersect_min']=test_pro_data['noOfIntersectingLaneLinesLeft_min']/test_pro_data['noOfIntersectingLaneLinesRight_min']
test_pro_data['ratio_Intersect_max']=test_pro_data['noOfIntersectingLaneLinesLeft_max']/test_pro_data['noOfIntersectingLaneLinesRight_max']

In [24]:
test_pro_data.columns

Index(['roadId', 'nunique', 'totalLaneLines_sum', 'totalLaneLines_mean',
       'totalLaneLines_min', 'totalLaneLines_max',
       'distFromLaneLineOnRight_mean', 'distFromLaneLineOnRight_min',
       'distFromLaneLineOnRight_sum', 'distFromLaneLineOnRight_max',
       'distFromLaneLineOnLeft_mean', 'distFromLaneLineOnLeft_min',
       'distFromLaneLineOnLeft_sum', 'distFromLaneLineOnLeft_max',
       'laneLineLength_mean', 'laneLineLength_min', 'laneLineLength_max',
       'laneLineLength_sum', 'roadLength_mean', 'roadLength_min',
       'roadLength_max', 'noOfIntersectingLaneLinesLeft_sum',
       'noOfIntersectingLaneLinesLeft_min',
       'noOfIntersectingLaneLinesLeft_max',
       'noOfIntersectingLaneLinesLeft_mean',
       'noOfIntersectingLaneLinesRight_sum',
       'noOfIntersectingLaneLinesRight_min',
       'noOfIntersectingLaneLinesRight_max', 'roadCategory',
       'isIntersectingWithRoadGeometry', 'laneLineCoordinates_count',
       'laneLineCoordinates_min', 'laneLineCoo

In [41]:
X=test_pro_data.columns.tolist()[1:]
Y='noOfLanes'
len(X)

40

In [26]:
train_pro_data=train_pro_data.fillna(0)

In [27]:
dtrain=xgb.DMatrix(train_pro_data[X],label=train_pro_data[Y])
dtest=xgb.DMatrix(test_pro_data[X])

def score_f1(preds,dtrain):
#     print ('>>>>',type(preds),preds.shape)
    labels=dtrain.get_label()
    return 'score_f1',metrics.f1_score(labels,preds,average='weighted')

param={}
param['eta']=.3
param['objective']='multi:softmax'
param['num_class']=6
param['max_depth']= 12
param['n_estimators']: 350
    
xcv=xgb.cv(param,dtrain,num_boost_round=200,feval=score_f1,maximize=True,early_stopping_rounds=50,\
       show_stdv=False,verbose_eval=10)


num_rounds=len(xcv)
num_rounds

watchlist=[(dtrain,'dtrain')]
model_xgb=xgb.train(param,dtrain,evals=watchlist,num_boost_round=num_rounds,feval=score_f1,maximize=True,\
                    early_stopping_rounds=50,\
       verbose_eval=10)

model_pred=model_xgb.predict(dtest)
print (np.unique(model_pred,return_counts=True))

model_pred=lbl.inverse_transform(model_pred.astype(int))
np.unique(model_pred)

sub_samp1=pd.DataFrame(test_pro_data['roadId'])
sub_samp1['noOfLanes']=model_pred

sub_samp1.to_csv(path+'1xgb_model_less_feature.csv',index=False)

[0]	train-score_f1:0.844042	test-score_f1:0.553058
[10]	train-score_f1:0.992186	test-score_f1:0.573735
[20]	train-score_f1:0.999896	test-score_f1:0.581709
[30]	train-score_f1:1	test-score_f1:0.583798
[40]	train-score_f1:1	test-score_f1:0.587413
[50]	train-score_f1:1	test-score_f1:0.588522
[60]	train-score_f1:1	test-score_f1:0.589599
[70]	train-score_f1:1	test-score_f1:0.59107
[80]	train-score_f1:1	test-score_f1:0.589944
[90]	train-score_f1:1	test-score_f1:0.589472
[100]	train-score_f1:1	test-score_f1:0.587852
[110]	train-score_f1:1	test-score_f1:0.586664
[120]	train-score_f1:1	test-score_f1:0.586846
[0]	train-score_f1:0.844042	test-score_f1:0.553058
[10]	train-score_f1:0.992186	test-score_f1:0.573735
[20]	train-score_f1:0.999896	test-score_f1:0.581709
[30]	train-score_f1:1	test-score_f1:0.583798
[40]	train-score_f1:1	test-score_f1:0.587413
[50]	train-score_f1:1	test-score_f1:0.588522
[60]	train-score_f1:1	test-score_f1:0.589599
[70]	train-score_f1:1	test-score_f1:0.59107
[80]	train-sco

In [28]:
train_X,test_X,train_y,test_y=cross_validation.train_test_split(train_pro_data[X],train_pro_data[Y],test_size=.3,random_state =74)
train_X.shape,train_y.shape,test_X.shape,test_y.shape

((3377, 40), (3377,), (1448, 40), (1448,))

### CAT

In [29]:
clf = CatBoostClassifier(learning_rate=0.01, iterations=1000, random_seed=0, \
                          loss_function='MultiClass', class_count=6)
clf.fit(train_X, train_y,eval_set=(test_X,test_y),use_best_model=True)

print(metrics.f1_score(test_y,np.ravel(clf.predict(test_X)),average='weighted'))
# clf.fit(train_pro_data[X], train_pro_data[Y])
pred_cat=np.ravel(clf.predict(test_pro_data[X]))

print (np.unique(pred_cat,return_counts=True))

pred_cat=lbl.inverse_transform(pred_cat.astype(int))
print(np.unique(pred_cat))

sub_samp2=pd.DataFrame(test_pro_data['roadId'])
sub_samp2['noOfLanes']=pred_cat

sub_samp2.to_csv(path+'1cat_model_less_feature.csv',index=False)

0.585240352056
(array([ 0.,  1.,  2.,  3.,  4.]), array([ 14, 215, 148,  21,   2], dtype=int64))
['1' '2' '3' '4' '5']


In [30]:
t4_params = {
    'boosting_type': 'gbdt', 'objective': 'multiclass', 'nthread': -1, 'silent': True,
    'num_leaves': 2**4, 'learning_rate': 0.05, 'max_depth': 4,
    'max_bin': 255, 'subsample_for_bin': 50000,
    'subsample': 0.8, 'subsample_freq': 1, 'colsample_bytree': 0.6, 'reg_alpha': 1, 'reg_lambda': 0,
    'min_split_gain': 0.5, 'min_child_weight': 1, 'min_child_samples': 10, 'scale_pos_weight': 1}

# they can be used directly to build a LGBMClassifier (which is wrapped in a sklearn fashion)

ligh_mdl=lgb.LGBMClassifier(**t4_params)

ligh_mdl.fit(train_X, train_y)

print(metrics.f1_score(test_y,ligh_mdl.predict(test_X),average='weighted'))
# ligh_mdl.fit(train_pro_data[X], train_pro_data[Y])
pred_lgb=ligh_mdl.predict(test_pro_data[X])
print(np.unique(pred_lgb,return_counts=True))

pred_lgb=lbl.inverse_transform(pred_lgb.astype(int))
print(np.unique(pred_lgb))

sub_samp3=pd.DataFrame(test_pro_data['roadId'])
sub_samp3['noOfLanes']=pred_lgb

sub_samp3.to_csv(path+'1lgb_model_less_feature.csv',index=False)

0.561789135616
(array([0, 1, 2, 3, 4], dtype=int64), array([ 21, 222, 133,  22,   2], dtype=int64))
['1' '2' '3' '4' '5']


In [31]:
xgb_train_res=(model_xgb.predict(dtrain).astype(int))
cat_train=np.ravel(clf.predict(train_pro_data[X]))
lgb_train=ligh_mdl.predict(train_pro_data[X])
new_train_data=pd.DataFrame(data={'xgb':xgb_train_res,'cat':cat_train,'lgb':lgb_train})

new_train_data.head()

,cat,lgb,xgb
0,2.0,2,1
1,1.0,1,1
2,2.0,2,1
3,2.0,2,2
4,2.0,2,2


In [32]:
xgb_test_res=(model_xgb.predict(dtest).astype(int))
cat_test=np.ravel(clf.predict(test_pro_data[X]))
lgb_test=ligh_mdl.predict(test_pro_data[X])
new_test_data=pd.DataFrame(data={'xgb':xgb_test_res,'cat':cat_test,'lgb':lgb_test})

new_test_data.head()

,cat,lgb,xgb
0,1.0,1,1
1,1.0,2,2
2,2.0,1,2
3,2.0,1,1
4,0.0,0,0


In [34]:
stack_train=pd.concat([train_pro_data,new_train_data],axis=1)
stack_train.columns

Index(['roadId', 'nunique', 'totalLaneLines_sum', 'totalLaneLines_mean',
       'totalLaneLines_min', 'totalLaneLines_max',
       'distFromLaneLineOnRight_mean', 'distFromLaneLineOnRight_min',
       'distFromLaneLineOnRight_sum', 'distFromLaneLineOnRight_max',
       'distFromLaneLineOnLeft_mean', 'distFromLaneLineOnLeft_min',
       'distFromLaneLineOnLeft_sum', 'distFromLaneLineOnLeft_max',
       'laneLineLength_mean', 'laneLineLength_min', 'laneLineLength_max',
       'laneLineLength_sum', 'roadLength_mean', 'roadLength_min',
       'roadLength_max', 'noOfIntersectingLaneLinesLeft_sum',
       'noOfIntersectingLaneLinesLeft_min',
       'noOfIntersectingLaneLinesLeft_max',
       'noOfIntersectingLaneLinesLeft_mean',
       'noOfIntersectingLaneLinesRight_sum',
       'noOfIntersectingLaneLinesRight_min',
       'noOfIntersectingLaneLinesRight_max', 'roadCategory',
       'isIntersectingWithRoadGeometry', 'laneLineCoordinates_count',
       'laneLineCoordinates_min', 'laneLineCoo

In [36]:
stack_test=pd.concat([test_pro_data,new_test_data],axis=1)
stack_test.columns

Index(['roadId', 'nunique', 'totalLaneLines_sum', 'totalLaneLines_mean',
       'totalLaneLines_min', 'totalLaneLines_max',
       'distFromLaneLineOnRight_mean', 'distFromLaneLineOnRight_min',
       'distFromLaneLineOnRight_sum', 'distFromLaneLineOnRight_max',
       'distFromLaneLineOnLeft_mean', 'distFromLaneLineOnLeft_min',
       'distFromLaneLineOnLeft_sum', 'distFromLaneLineOnLeft_max',
       'laneLineLength_mean', 'laneLineLength_min', 'laneLineLength_max',
       'laneLineLength_sum', 'roadLength_mean', 'roadLength_min',
       'roadLength_max', 'noOfIntersectingLaneLinesLeft_sum',
       'noOfIntersectingLaneLinesLeft_min',
       'noOfIntersectingLaneLinesLeft_max',
       'noOfIntersectingLaneLinesLeft_mean',
       'noOfIntersectingLaneLinesRight_sum',
       'noOfIntersectingLaneLinesRight_min',
       'noOfIntersectingLaneLinesRight_max', 'roadCategory',
       'isIntersectingWithRoadGeometry', 'laneLineCoordinates_count',
       'laneLineCoordinates_min', 'laneLineCoo

In [46]:
stack_test.dtypes.value_counts()

float64    26
int64      16
int32       1
object      1
dtype: int64

In [47]:
stack_train.dtypes.value_counts()

float64    26
int64      17
int32       1
object      1
dtype: int64

In [37]:
new_X=stack_test.columns[1:]
Y='noOfLanes'
len(new_X)

43

In [51]:
print (stack_train[new_X].dtypes,stack_train[Y].shape)

nunique                                 int64
totalLaneLines_sum                      int64
totalLaneLines_mean                   float64
totalLaneLines_min                      int64
totalLaneLines_max                      int64
distFromLaneLineOnRight_mean          float64
distFromLaneLineOnRight_min           float64
distFromLaneLineOnRight_sum           float64
distFromLaneLineOnRight_max           float64
distFromLaneLineOnLeft_mean           float64
distFromLaneLineOnLeft_min            float64
distFromLaneLineOnLeft_sum            float64
distFromLaneLineOnLeft_max            float64
laneLineLength_mean                   float64
laneLineLength_min                    float64
laneLineLength_max                    float64
laneLineLength_sum                    float64
roadLength_mean                       float64
roadLength_min                        float64
roadLength_max                        float64
noOfIntersectingLaneLinesLeft_sum       int64
noOfIntersectingLaneLinesLeft_min 

In [49]:
dtrain=xgb.DMatrix(stack_train[new_X],label=stack_train[Y])
dtest=xgb.DMatrix(stack_test[new_X])

def score_f1(preds,dtrain):
#     print ('>>>>',type(preds),preds.shape)
    labels=dtrain.get_label()
    return 'score_f1',metrics.f1_score(labels,preds,average='weighted')

param={}
param['eta']=.3
param['objective']='multi:softmax'
param['num_class']=6
param['max_depth']= 12
param['n_estimators']: 350
    
xcv=xgb.cv(param,dtrain,num_boost_round=200,feval=score_f1,maximize=True,early_stopping_rounds=50,\
       show_stdv=False,verbose_eval=10)

[0]	train-score_f1:1	test-score_f1:1
[10]	train-score_f1:1	test-score_f1:1
[20]	train-score_f1:1	test-score_f1:1
[30]	train-score_f1:1	test-score_f1:1
[40]	train-score_f1:1	test-score_f1:1


In [53]:
num_rounds=len(xcv)
num_rounds

watchlist=[(dtrain,'dtrain')]
model_xgb=xgb.train(param,dtrain,evals=watchlist,num_boost_round=num_rounds,feval=score_f1,maximize=True,\
                    early_stopping_rounds=50,\
       verbose_eval=10)

[0]	dtrain-score_f1:1
Will train until dtrain-score_f1 hasn't improved in 50 rounds.


In [54]:
model_pred=model_xgb.predict(dtest)
print (np.unique(model_pred,return_counts=True))

(array([ 0.,  1.,  2.,  3.,  4.,  5.], dtype=float32), array([ 15, 224, 123,  31,   5,   2], dtype=int64))


In [56]:
model_pred[:30]

array([ 1.,  2.,  2.,  1.,  0.,  0.,  1.,  2.,  3.,  2.,  3.,  2.,  2.,
        2.,  1.,  2.,  1.,  1.,  1.,  1.,  2.,  1.,  3.,  1.,  2.,  1.,
        1.,  1.,  1.,  1.], dtype=float32)

In [52]:
model_pred=lbl.inverse_transform(model_pred.astype(int))
np.unique(model_pred)

sub_samp1=pd.DataFrame(stack_test['roadId'])
sub_samp1['noOfLanes']=model_pred

sub_samp1.to_csv(path+'1xgb_model_less_feature.csv',index=False)

[0]	dtrain-score_f1:1
Will train until dtrain-score_f1 hasn't improved in 50 rounds.
(array([ 0.,  1.,  2.,  3.,  4.,  5.], dtype=float32), array([ 15, 224, 123,  31,   5,   2], dtype=int64))


In [ ]:
def mode_data(x,y,z):
    if x==y:
        return x
    elif y==z:
        return y
    else:
        return 'lala'
def val_repl(x,y):
    if x=='lala':
        return y
    else:
        return x
    
lolwa=pd.DataFrame(data={'roadId':test_pro_data['roadId'],'xgb':model_pred,'cat':pred_cat,'lgb':pred_lgb})
lolwa['ense']=lolwa[['xgb','cat','lgb']].apply(lambda x: mode_data(*x),axis=1)
lolwa['ense']=lolwa[['ense','xgb']].apply(lambda x: val_repl(*x),axis=1)

lolwa=lolwa[['roadId','ense']]
lolwa.columns=['roadId','noOfLanes']

print (np.unique(lolwa['noOfLanes'],return_counts=True))

lolwa.to_csv('1ense_xcl.csv',index=False)